* 1ero importamos los paquetes

In [48]:
import pandas as pd 
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV



* Cargar los datos

In [49]:
# Cargar los datos train
train_data  = pd.read_csv('data/train.csv')

# Cargar los datos test
test_data = pd.read_csv('data/test.csv')

* Explorar el conjunto de datos de entrenamiento

In [50]:
train_data.head()

,ID,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
0,9580,0.668999,58,2,0.449504,3425.0,9,1,1,1,1.0,0
1,39755,0.015922,71,0,6.000000,NaN,5,0,0,0,0.0,0
2,118799,0.183062,52,1,0.035593,5000.0,9,0,0,0,0.0,0
3,16489,0.162301,77,0,0.227886,2000.0,8,0,0,0,0.0,0
4,149857,0.404199,30,0,0.026010,5843.0,4,0,0,0,0.0,0


In [51]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   ID                                    105000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  105000 non-null  float64
 2   Age                                   105000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  105000 non-null  int64  
 4   DebtRatio                             105000 non-null  float64
 5   MonthlyIncome                         84164 non-null   float64
 6   NumberOfOpenCreditLinesAndLoans       105000 non-null  int64  
 7   NumberOfTimes90DaysLate               105000 non-null  int64  
 8   NumberRealEstateLoansOrLines          105000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  105000 non-null  int64  
 10  NumberOfDependents                    102236 non-null  float64
 11  

In [52]:
train_data.describe()

,ID,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
count,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,8.416400e+04,105000.000000,105000.000000,105000.000000,105000.000000,102236.000000,105000.000000
mean,75006.458152,5.378324,52.325610,0.409352,352.044192,6.703641e+03,8.459952,0.254619,1.019010,0.228762,0.757933,0.066514
std,43315.742022,201.573457,14.766425,4.056717,1820.229318,1.622288e+04,5.134329,4.032506,1.131065,4.017864,1.115273,0.249180
min,1.000000,0.000000,21.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37590.750000,0.029974,41.000000,0.000000,0.174950,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,74941.500000,0.154252,52.000000,0.000000,0.366061,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,112542.500000,0.556035,63.000000,0.000000,0.870083,8.250000e+03,11.000000,0.000000,2.000000,0.000000,1.000000,0.000000
max,149999.000000,29110.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000,1.000000


* Distribución de la variable objetivo

In [53]:
train_data['SeriousDlqin2yrs'].value_counts() 

SeriousDlqin2yrs
0    98016
1     6984
Name: count, dtype: int64

* Explorar el conjunto de datos de prueba

In [54]:
test_data.head()

,ID,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,129460,1.000000,21,0,8.000000,NaN,0,0,0,0,NaN
1,134018,0.009878,38,0,0.229978,10500.0,10,0,1,0,1.0
2,86523,0.276836,70,0,1914.000000,NaN,23,0,1,0,0.0
3,138466,0.045413,75,0,452.000000,NaN,4,0,0,0,0.0
4,143905,0.000000,82,0,0.000000,NaN,5,0,0,0,0.0


In [55]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID                                    45000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  45000 non-null  float64
 2   Age                                   45000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  45000 non-null  int64  
 4   DebtRatio                             45000 non-null  float64
 5   MonthlyIncome                         36105 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       45000 non-null  int64  
 7   NumberOfTimes90DaysLate               45000 non-null  int64  
 8   NumberRealEstateLoansOrLines          45000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  45000 non-null  int64  
 10  NumberOfDependents                    43840 non-null  float64
dtypes: float64(4), 

* Identificación de problemas y preparación, valores faltantes

In [56]:
# Conteo de valores faltantes en el conjunto de entrenamiento
missing_train = train_data.isnull().sum()  
missing_train[missing_train > 0]

MonthlyIncome         20836
NumberOfDependents     2764
dtype: int64

In [57]:
# Conteo de valores faltantes en el conjunto de prueba
missing_test = test_data.isnull().sum()    
missing_test[missing_test > 0]

MonthlyIncome         8895
NumberOfDependents    1160
dtype: int64

* Revisión de la estructura

In [58]:
# Columnas en el Conjunto de Datos de Entrenamiento
train_data.columns

Index(['ID', 'RevolvingUtilizationOfUnsecuredLines', 'Age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents', 'SeriousDlqin2yrs'],
      dtype='object')

In [59]:
# Columnas en el Conjunto de Datos de Prueba
test_data.columns

Index(['ID', 'RevolvingUtilizationOfUnsecuredLines', 'Age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

* Limpieza de Datos / Rellenar valores nulos

In [60]:
# Rellenar valores nulos en MonthlyIncome y NumberOfDependents en el conjunto de entrenamiento
train_data['MonthlyIncome'].fillna(train_data['MonthlyIncome'].median(), inplace=True)
train_data['NumberOfDependents'].fillna(train_data['NumberOfDependents'].median(), inplace=True)

In [61]:
# Rellenar valores nulos en MonthlyIncome y NumberOfDependents en el conjunto de prueba
test_data['MonthlyIncome'].fillna(test_data['MonthlyIncome'].median(), inplace=True)
test_data['NumberOfDependents'].fillna(test_data['NumberOfDependents'].median(), inplace=True)

In [62]:
train_data[['MonthlyIncome', 'NumberOfDependents']].isnull().sum()

MonthlyIncome         0
NumberOfDependents    0
dtype: int64

In [63]:
test_data[['MonthlyIncome', 'NumberOfDependents']].isnull().sum()

MonthlyIncome         0
NumberOfDependents    0
dtype: int64

* Normalización/Estandarización

In [64]:
# Seleccionar características numéricas para estandarización
num_features = ['MonthlyIncome', 'NumberOfDependents']

# Crear el escalador estándar
scaler = StandardScaler()

In [65]:
# Ajustar el escalador en el conjunto de entrenamiento y transformar los datos
train_data[num_features] = scaler.fit_transform(train_data[num_features])

train_data[num_features].head()

,MonthlyIncome,NumberOfDependents
0,-0.207791,0.236659
1,-0.071899,-0.666553
2,-0.099422,-0.666553
3,-0.305840,-0.666553
4,-0.041418,-0.666553


In [66]:
# Transformar los datos del conjunto de prueba usando el escalador ajustado en el entrenamiento
test_data[num_features] = scaler.transform(test_data[num_features])

test_data[num_features].head()

,MonthlyIncome,NumberOfDependents
0,-0.071899,-0.666553
1,0.279013,0.236659
2,-0.071899,-0.666553
3,-0.071899,-0.666553
4,-0.071899,-0.666553


* División en Conjunto de Entrenamiento y Conjunto de Validación

In [67]:
# Separar características y etiquetas

# Dividir el conjunto de datos de entrenamiento en conjunto de entrenamiento y conjunto de validación
X = train_data.drop(columns=['SeriousDlqin2yrs'])
y = train_data['SeriousDlqin2yrs']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

* Construcción del Modelo

In [68]:
# Inicializar el modelo de regresión logística
model = LogisticRegression(solver='liblinear', random_state=42)

* Entrenamiento del Modelo

In [69]:
# Entrenar el modelo
model.fit(X_train, y_train)

LogisticRegression(random_state=42, solver='liblinear')

* Evaluación del Modelo

In [70]:

# Predecir en el conjunto de validación
y_valid_pred = model.predict_proba(X_valid)[:, 1]

# Evaluar el modelo utilizando la métrica AUC
auc_score = roc_auc_score(y_valid, y_valid_pred)
print(f"AUC Score: {auc_score:.4f}")

AUC Score: 0.6320


* Ajuste de Hiperparámetros

In [71]:
# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

# Configurar GridSearchCV
grid_search = GridSearchCV(LogisticRegression(solver='liblinear', random_state=42), param_grid, cv=5, scoring='roc_auc')

# Ajustar GridSearchCV
grid_search.fit(X_train, y_train)

# Mostrar los mejores parámetros
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best AUC Score: {grid_search.best_score_:.4f}")

# Usar el mejor modelo
best_model = grid_search.best_estimator_


Best Parameters: {'C': 100, 'penalty': 'l1'}
Best AUC Score: 0.6982


In [72]:

# Realizar validación cruzada
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='roc_auc')

# Mostrar la puntuación media de la validación cruzada
print(f"AUC Media en validación cruzada: {cv_scores.mean():.4f}")


AUC Media en validación cruzada: 0.6982


* Predicción

In [73]:
test_data

,ID,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,129460,1.000000,21,0,8.000000,-0.071899,0,0,0,0,-0.666553
1,134018,0.009878,38,0,0.229978,0.279013,10,0,1,0,0.236659
2,86523,0.276836,70,0,1914.000000,-0.071899,23,0,1,0,-0.666553
3,138466,0.045413,75,0,452.000000,-0.071899,4,0,0,0,-0.666553
4,143905,0.000000,82,0,0.000000,-0.071899,5,0,0,0,-0.666553
...,...,...,...,...,...,...,...,...,...,...,...
44995,124596,0.055997,70,0,51.000000,-0.071899,2,0,0,0,-0.666553
44996,75895,1.000000,62,0,2796.000000,-0.071899,5,0,1,0,2.043082
44997,92453,0.673065,56,1,0.511132,0.072594,9,0,2,0,2.946294
44998,139288,1.000000,22,0,0.000000,-0.271437,0,0,0,0,-0.666553


In [74]:
X_test = test_data.drop(columns=['ID'])

In [75]:
X_test

,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1.000000,21,0,8.000000,-0.071899,0,0,0,0,-0.666553
1,0.009878,38,0,0.229978,0.279013,10,0,1,0,0.236659
2,0.276836,70,0,1914.000000,-0.071899,23,0,1,0,-0.666553
3,0.045413,75,0,452.000000,-0.071899,4,0,0,0,-0.666553
4,0.000000,82,0,0.000000,-0.071899,5,0,0,0,-0.666553
...,...,...,...,...,...,...,...,...,...,...
44995,0.055997,70,0,51.000000,-0.071899,2,0,0,0,-0.666553
44996,1.000000,62,0,2796.000000,-0.071899,5,0,1,0,2.043082
44997,0.673065,56,1,0.511132,0.072594,9,0,2,0,2.946294
44998,1.000000,22,0,0.000000,-0.271437,0,0,0,0,-0.666553


* Preparación de Resultados

In [76]:
# Hacer predicciones en el conjunto de prueba
test_probabilities = best_model.predict_proba(X_test)[:, 1]

# Convertir las probabilidades en clases binarias usando un umbral de 0.5
test_predictions = (test_probabilities >= 0.5).astype(int)


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- ID


In [ ]:
# Formateo del Archivo CSV
# Crear un DataFrame con las predicciones
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SeriousDlqin2yrs': test_predictions
})

In [ ]:
print(results.groupby('SeriousDlqin2yrs').size())

SeriousDlqin2yrs
0    44810
1      190
dtype: int64


* Guardar el Archivo CSV

In [ ]:
# Guardar el archivo CSV
results.to_csv('submission.csv', index=False)
